# Decompositions Playground

Given an integer number, $target$, we want to get all the possible combinations of at most $k$ smaller _standard_ numbers that sum up to the target number. However this problem is just a special case of the problem of finding all the possible combinations of a set of given numbers so, let start calculating the universe of all the possible combinations.

# The Algorithm

Starting with an empty set ${ }$, we use it as a template to be extended by each of the _standard denominations_ smaller or equal than the tail element of the template set and as result we obtains a new set of template sets. For example:

```
take {} and extend:
{10}
{9}
{8}
{6}
... (continue)
```

Then we repeat the process for each new template set, this means, take a template set and extend it with each of the _standard denominations_. For example:

```
take {10} and extend
{10, 10}
{10, 9}
{10, 8}
{10, 6}
.... (continue)
take {9} and extend
{9, 9}
{9, 8}
{9, 6}
... (and so on)
```

Basically it is a recursive process that builds the new template sets by appending each of the _standard denominations_, smaller or equal than the tail element of the template set, to each of the template sets.

# Implementation

Lets calculate the the decompositions of $k = 3$ and $std_denominations = [1, 2, 3, 4, 5, 6, 8, 9, 10]$

In [85]:
// Parameters
let denoms = [| 10; 9; 8; 6; 5; 4; 3; 2; 1 |]

// Other parameters we will use much later
let target = 21
let tolerance = 1

But before start let create a helper method to print all the decompositions. This will help us to reduce the noise in our code (c# is already too noisy for this kind of tasks)

In [86]:
#!fsharp
let printx (sequencex: seq<int list>) : unit =
    for s in sequencex do
        printfn "Sum: %d %A" (s |> Seq.sum) s

And this is the *code*:

In [87]:
let rec Comb1 acc k denoms : seq<int list> =
    let acc = (denoms |> Array.head) :: acc
    if k = 0
        then seq { acc }
        else seq { 0..denoms.Length-1 } |> Seq.collect (fun i -> Comb1 acc (k - 1) denoms.[ i.. ])

printx (Comb1 [] 2 denoms)

Sum: 30 [10; 10; 10]
Sum: 29 [9; 10; 10]
Sum: 28 [8; 10; 10]
Sum: 26 [6; 10; 10]
Sum: 25 [5; 10; 10]
Sum: 24 [4; 10; 10]
Sum: 23 [3; 10; 10]
Sum: 22 [2; 10; 10]
Sum: 21 [1; 10; 10]
Sum: 28 [9; 9; 10]
Sum: 27 [8; 9; 10]
Sum: 25 [6; 9; 10]
Sum: 24 [5; 9; 10]
Sum: 23 [4; 9; 10]
Sum: 22 [3; 9; 10]
Sum: 21 [2; 9; 10]
Sum: 20 [1; 9; 10]
Sum: 26 [8; 8; 10]
Sum: 24 [6; 8; 10]
Sum: 23 [5; 8; 10]
Sum: 22 [4; 8; 10]
Sum: 21 [3; 8; 10]
Sum: 20 [2; 8; 10]
Sum: 19 [1; 8; 10]
Sum: 22 [6; 6; 10]
Sum: 21 [5; 6; 10]
Sum: 20 [4; 6; 10]
Sum: 19 [3; 6; 10]
Sum: 18 [2; 6; 10]
Sum: 17 [1; 6; 10]
Sum: 20 [5; 5; 10]
Sum: 19 [4; 5; 10]
Sum: 18 [3; 5; 10]
Sum: 17 [2; 5; 10]
Sum: 16 [1; 5; 10]
Sum: 18 [4; 4; 10]
Sum: 17 [3; 4; 10]
Sum: 16 [2; 4; 10]
Sum: 15 [1; 4; 10]
Sum: 16 [3; 3; 10]
Sum: 15 [2; 3; 10]
Sum: 14 [1; 3; 10]
Sum: 14 [2; 2; 10]
Sum: 13 [1; 2; 10]
Sum: 12 [1; 1; 10]


# Compute only those sets that sum up to a target value (or less)

The trick here is to limit the set of _standard denominations_ the algorithm can use in each iteration. For example, imagine the target is $25$:

| target | set            | sum  | remaining | standard denominations available |
|-------:|---------------:|-----:|----------:|---------------------------------:|
|     25 |         { 10 } |   10 |        15 |  { 10, 9 , 8, 6, 5, 4, 3, 2, 1 } |
|     15 |     { 10, 10 } |   20 |         5 |                { 5, 4, 3, 2, 1 } |
|      5 | { 10, 10,  5 } |   25 |         0 |                              { } |

Below we have the code that filters those values that are too big to be explored.

In [88]:
let rec Comb2 target acc k denoms : seq<int list> =
    let cur = denoms |> Array.head
    let acc = cur :: acc
    let remaining = target - cur
    if k = 0
        then seq { acc }
        else 
            let candidates = denoms |> Array.skipWhile (fun x -> x > remaining)
            seq { 0..candidates.Length-1 } |> Seq.collect (fun i -> Comb2 remaining acc (k - 1) candidates.[ i.. ])

printx (Comb2 21 [] 2 denoms)

Sum: 21 [1; 10; 10]
Sum: 21 [2; 9; 10]
Sum: 20 [1; 9; 10]
Sum: 21 [3; 8; 10]
Sum: 20 [2; 8; 10]
Sum: 19 [1; 8; 10]
Sum: 21 [5; 6; 10]
Sum: 20 [4; 6; 10]
Sum: 19 [3; 6; 10]
Sum: 18 [2; 6; 10]
Sum: 17 [1; 6; 10]
Sum: 20 [5; 5; 10]
Sum: 19 [4; 5; 10]
Sum: 18 [3; 5; 10]
Sum: 17 [2; 5; 10]
Sum: 16 [1; 5; 10]
Sum: 18 [4; 4; 10]
Sum: 17 [3; 4; 10]
Sum: 16 [2; 4; 10]
Sum: 15 [1; 4; 10]
Sum: 16 [3; 3; 10]
Sum: 15 [2; 3; 10]
Sum: 14 [1; 3; 10]
Sum: 14 [2; 2; 10]
Sum: 13 [1; 2; 10]
Sum: 12 [1; 1; 10]


This is better because it doesn't generate lots and lots of unnecessary combinations but it still has one problems and it is that it still generates lots and lots of useless combinations because their sum is too far from the expected target. It is important to abort the generation of those combinations once we know they are not possible.

For example, imagine we want to generate decompositions of $k=3$ that sum up to 12 using our previously define _standard denominations_. Imagine we are generating the first set:

| target | set            | sum  | remaining | denominations available | can continue? |
|-------:|---------------:|-----:|----------:|---------------------------------:|---------------
|     12 |         { 10 } |   10 |         2 |  { 2, 1 } | Yes. Because there are space for 2 extra elements and 2 * 2 >= 2 (remaining). |
|     12 |     {  6,  3 } |    9 |         3 |  { 3, 2, 1 } | Yes. Because there is space for 1 extra elements and 1 * 3 >= 3 (remaining).
|     12 |     {  5,  3 } |    8 |         4 |  { 3, 2, 1 } | No. Because there is space for 1 extra elements and 1 * 3 < 4 (remaining).

This is much much better because now we get the only 8 decompositions that sum up to 21. However, not all numbers can be expressed as the sum of other smaller positive numbers. Imagine our _standard denominations_ are just ${ 100, 50, 25, 10, 5 }$, in this case it is impossible to represent the number $21$.

For this reason we need to introduce the concept of _tolerance_, defined as the acceptable difference between the expected target and the one obteined as the result of the sum of the combination individual elements.

# Do not explore further once a much is find

This version looks better, right? But if take a look at the 3rd result ($[10, 9, 1]$ = 20) we should ask why!? What's the point of that combination? Clearly if we explore the combinations in expanding the template (accumulator) with standard denominations in descending order then, it is silly to continue searching for combinations once we have already found one exact much because all the subsecuent combinations will sum less.

We will need a linq extension to make the code look easier. `TakeUnti` is a method that takes a sequence and returns a sequence of elements until a condition is met with the first element that meets the condition included.

In [89]:
let rec combination currentDenomination localTarget tolerance accumulator k denoms =
    let acc = currentDenomination :: accumulator
    let remaining = localTarget - currentDenomination
    if k = 0 || remaining < tolerance
        then seq { acc }
        else
            let nextDenoms = denoms |> Array.filter (fun x-> x <= remaining && k * x >= (remaining - tolerance))
            seq { 0..nextDenoms.Length-1 } |> Seq.collect (fun i -> combination nextDenoms remaining tolerance acc (k - 1) nextDenoms.[ i.. ])


Unhandled Exception: input.fsx (8,92)-(8,101) typecheck error The type 'int []' does not match the type 'int'

In [ ]:
IEnumerable<ImmutableList<long>> CombinationsUptoAndPruneWithTolerance2(long currentDenomination, long target, long localTarget, long tolerance, ImmutableList<long> accumulator, int k, long[] denoms)
{
	accumulator = accumulator.Add(currentDenomination);
	var remaining = localTarget - currentDenomination;
	if (k == 0 || remaining < tolerance)
		return new[] { accumulator };
	
	denoms = denoms.Where(x => x <= remaining).ToArray();
	return denoms
		.TakeWhile(std => k * std >= remaining - tolerance)
		.SelectMany((std, i) => 
			CombinationsUptoAndPruneWithTolerance2(std, target, remaining, tolerance, accumulator, k - 1, denoms[i..])
			.TakeUntil(x => x.Sum() == target));
}

Print(denoms.SelectMany((std, i) => CombinationsUptoAndPruneWithTolerance2(std, target, target, tolerance, initialState, k - 1, denoms[i..])));

# Optimization (first part)

In all previous code snippets we have used an `ImmutableList` as accumulator bacause it makes an efficient usage of memory given it is implemented with Avl trees. However, we want to reduce the heap allocations and total memory usage as much as possible what is doable if we introduce some assumptions:

* $k$ will never be greater than $8$ and,
* $len(standarddenoms)$ will never be larger than $2^8$

With this we can use an `ulong` to store the indexes of the _standard denomination_ instead of their values. 



In [ ]:
IEnumerable<(long Sum, ulong Decomposition)> CompactCombinationsOfIndexesUptoAndPruneWithTolerance(int currentDenominationIdx, long target, long localTarget, long tolerance, ulong accumulator, long sum, int k, long[] denoms)
{
	accumulator = (accumulator << 8) | ((ulong)currentDenominationIdx & 0xff);
	var currentDenomination = denoms[currentDenominationIdx];
	sum += currentDenomination;
	var remaining = localTarget - currentDenomination;
	if (k == 0 || remaining < tolerance)
		return new[] { (sum, accumulator) };

	currentDenominationIdx += denoms.Skip(currentDenominationIdx).Count(x => x > remaining);
	return Enumerable.Range(0, denoms.Length - currentDenominationIdx)
		.TakeWhile(i => k * denoms[currentDenominationIdx + i] >= remaining - tolerance)
		.SelectMany((_, i) => 
			CompactCombinationsOfIndexesUptoAndPruneWithTolerance(currentDenominationIdx + i, target, remaining, tolerance, accumulator, sum, k - 1, denoms)
			.TakeUntil(x => x.Sum == target));
}



In [ ]:
void Print(IEnumerable<(long Sum, ulong Decomposition)> decompositions)
{
	foreach (var (composition, row) in decompositions.Select((x,r) => (x, r)))
	{
		var indexes = BitConverter.GetBytes(composition.Decomposition);
		var remaining = composition.Sum;
		var i = 0;
		var list = new List<long>();
		do
		{
			var val = denoms[indexes[i++]];
			list.Insert(0, val);
			remaining -= val;
		}while(remaining > 0);
		Console.WriteLine("#{0} Sum: {1} -> [{2}]", row, composition.Sum, string.Join(", ", list));
	}
}

In [ ]:
Print(denoms.SelectMany((_, i) => CompactCombinationsOfIndexesUptoAndPruneWithTolerance(i, target, target, tolerance, 0ul, 0, k - 1, denoms)));

In [ ]:
IEnumerable<(long Sum, ulong Decomposition)> cx(int currentDenominationIdx, long target, long localTarget, long tolerance, ulong accumulator, long sum, int k, long[] denoms)
{
	accumulator = (accumulator << 8) | ((ulong)currentDenominationIdx & 0xff);
	var currentDenomination = denoms[currentDenominationIdx];
	sum += currentDenomination;
	var remaining = localTarget - currentDenomination;
	if (k == 0 || remaining < tolerance)
		return new[] { (sum, accumulator) };

	currentDenominationIdx += denoms.Skip(currentDenominationIdx).Count(x => x > remaining);
	return Enumerable.Range(0, denoms.Length - currentDenominationIdx)
		.TakeWhile(i => k * denoms[currentDenominationIdx + i] >= remaining - tolerance)
		.SelectMany((_, i) => 
			cx(currentDenominationIdx + i, target, remaining, tolerance, accumulator, sum, k - 1, denoms)
			.TakeUntil(x => x.Sum == target));
}


Print(denoms.SelectMany((_, i) => cx(i, target, target, tolerance, 0ul, 0, k - 1, denoms)));


We have now a higher quality result set but the code is a bit hard to follow basically because of the long list of parameters but that can be improved easily. Let start by extracting those that are constant.

In [103]:
let combx target tolerance maxlength (denoms: int[]) =
    let rec icombx currentDenominationIdx accumulator sum k =
        let new_acc = (accumulator <<< 8) ||| (currentDenominationIdx &&& 0xff)
        let new_sum = sum + denoms.[ currentDenominationIdx ]
        let remaining = target - new_sum
        if k = 0 || remaining < tolerance
            then seq { (new_sum, new_acc) }
            else
                let new_denominationIdx = currentDenominationIdx + (denoms.[currentDenominationIdx..] |> Array.filter (fun x -> x > remaining) |> Array.length)
                seq { 0..(denoms.Length - new_denominationIdx - 1) }
                |> Seq.takeWhile (fun i -> k * denoms.[new_denominationIdx + i] >= remaining - tolerance)
                |> Seq.collect (fun i -> icombx (new_denominationIdx + i) new_acc new_sum (k - 1))

    seq { 0..denoms.Length-1 } |> Seq.collect (fun i -> icombx i 0 0 (maxlength - 1))

(combx 21 1 3 denoms)

index,Item1,Item2
0,21,8
1,21,263
2,20,264
3,21,518
4,20,519
5,21,772
6,20,773
7,20,1028
8,21,65798
9,20,65799
